In [1]:
!pip install bertopic
import pandas as pd
import re
import nltk
from bertopic import BERTopic
import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 96.0 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 86.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 81.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 103.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 43.2 MB/s eta 0:00:0000:0100:01
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp39-cp39-linux_x86_64.whl size=3584141 sha256=39e824847361862f5df09dad8072c42439c3ef9a2625d5e14462fa8604d63204
  Stored in directory: /

/usr/local/lib/python3.9/dist-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.9/dist-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/usr/local/lib/python3.9/dist-packages/umap/distances.py:1086: NumbaDeprecationWarning: The 'nopython' keyword argument 

GPU: NVIDIA A100-SXM4-80GB


## Data

In [30]:
import os
os.listdir('../datasets/kickstarter_cleaned')

[]

In [ ]:
df_kickstarter = pd.read_csv('../datasets/kickstarter_cleaned/kickstarter_cleaned_topicmodelling.csv')

In [ ]:
df_kickstarter = df_kickstarter[df_kickstarter['num_words'] <= 2000]

In [ ]:
df_kickstarter.to_csv('../datasets/kickstarter_cleaned/kickstarter_cleaned_topicmodelling.csv')

# Functions

In [ ]:
# Usage optional -> probably better embeddings without this
def preprocess_text(document: str) -> str:

    # Remove all the special characters
    document = re.sub(r'\W', ' ', document)

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Converting to Lowercase
    document = document.lower()

    return document;

# BERTopic

In [42]:
# Copy Dataframe
df_bertopic_kickstarter = df_kickstarter.copy()

In [43]:
# Remove the columns
df_bertopic_kickstarter.drop(df_bertopic_kickstarter.columns.difference(['project_description', 'project_category_id', 'project_parent_category_id', 'project_state', 'project_launched_at']), axis=1,inplace=True)

### Technology Category

In [44]:
df_bertopic_kickstarter_technology = df_bertopic_kickstarter.copy()

In [45]:
# Filter Dataframe by Technology category (number 16)
df_bertopic_kickstarter_technology = df_bertopic_kickstarter_technology[(df_bertopic_kickstarter_technology['project_category_id'] == 16) | (df_bertopic_kickstarter_technology['project_parent_category_id'] == 16)]

In [46]:
docs = df_bertopic_kickstarter_technology['project_description'].tolist()

In [ ]:
from sentence_transformers import SentenceTransformer

# 512 token limit default is all-MiniLM-L6-v2 with 256 token limit
sentence_model = SentenceTransformer("all-distilroberta-v1")
embeddings = sentence_model.encode(docs, show_progress_bar=True)

In [8]:
from bertopic.vectorizers import ClassTfidfTransformer
from hdbscan import HDBSCAN

# Only Topics with 15 documents or more
hdbscan_model = HDBSCAN(min_cluster_size=15)

# removes noise
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=(['0000', 'with sound', 'play replay', 'replay with', 'capable browser', 'sound 0000', 'html5 capable', 'content play', '0000 0000', 'sound play']))

# removes stopwords and other extremes
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True) # vectorizer_model = CountVectorizer(stop_words="english") as an alternative to remove stopwords (does not remove other extremes)

topic_model = BERTopic(hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model, language="english", calculate_probabilities=True, verbose=True, n_gram_range=(1, 2)) #embedding_model=sentence_model
topics, probs = topic_model.fit_transform(docs) #add embeddings as parameter to use costum embeddings model

Batches:   0%|          | 0/843 [00:00<?, ?it/s]

2023-09-13 14:38:32,417 - BERTopic - Transformed documents to Embeddings
2023-09-13 14:39:10,944 - BERTopic - Reduced dimensionality


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-09-13 14:40:36,726 - BERTopic - Clustered reduced embeddings


### Visualization of Topics

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=300)

In [ ]:
topic_model.visualize_barchart(top_n_topics=300)

In [12]:
freq = topic_model.get_topic_info(); freq.head(300)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,11012,-1_my_project_phone_would,"[my, project, phone, would, work, people, will...",[I (Michael) have been a photography instructo...
1,0,638,0_light_led_lighting_lights,"[light, led, lighting, lights, leds, the light...",[Tired of tripping over things in the dark at ...
2,1,497,1_social_social media_events_event,"[social, social media, events, event, travel, ...",[IMPORTANT UPDATE: Our campaign has moved! Ch...
3,2,487,2_security_files_encryption_passwords,"[security, files, encryption, passwords, passw...","[We re-invented how your devices work Today, m..."
4,3,481,3_golf_sports_swing_players,"[golf, sports, swing, players, ball, athletes,...",[I'm on a mission to turn analytics into a too...
...,...,...,...,...,...
280,279,10,279_touch easel_easel_touch_playsurface,"[touch easel, easel, touch, playsurface, easyt...",[Sonzia presents Touch Easel: The School Desk ...
281,280,10,280_komunity_amal_and neil_life groups,"[komunity, amal, and neil, life groups, neil, ...",[Kindness Begins with Kom-Unity\n PLAY\n\nThe ...
282,281,10,281_paint_paint can_roller covers_the paint,"[paint, paint can, roller covers, the paint, r...",[ Product Info:The Paint Can Holder was design...
283,282,10,282_moto_tramigo_swagon_tramigo moto,"[moto, tramigo, swagon, tramigo moto, moto clo...","[FINALLY, A PRO-ACTIVE WAY TO HAVE A SAFER BIK..."


In [19]:
topic_model.get_topic(15, full=True)

{'Main': [('0000', 0.14969733203398805),
  ('with sound', 0.14967371210649202),
  ('play replay', 0.13543283793945604),
  ('replay with', 0.1354268425235859),
  ('capable browser', 0.13542084797498574),
  ('sound 0000', 0.13542084797498574),
  ('html5 capable', 0.13542084797498574),
  ('content play', 0.13541485429341663),
  ('0000 0000', 0.13541485429341663),
  ('sound play', 0.13537292277925492)]}

In [15]:
topic_model.get_representative_docs(15)

['So What Is Movyabox? Movyabox is a self-contained, portable streaming media server which allows users to easily load, take and stream their media collections (including both music and video content) at home or on the move. Simply put, a media server is a device that stores and shares media content of multiple types. Movyabox allows the user to play their content on up to six devices regardless of the operating platform of the device (smartphone, tablet, PC or Mac).         While at their homes, users can connect Movyabox to their TV, allowing them to watch streamed content from such sites as Netflix, HBO, and others, but also from content that has been downloaded to the device.         Movyabox at Home         You\'ll need an HTML5 capable browser to see this content.       Play                                   Replay with sound          Play with \r\n sound             00:00   00:00                                       Some additional screen shots from our TV interface    courtesy

### Dynamic Topic Modelling

In [25]:
from datetime import datetime

In [57]:
timestamps  = pd.to_datetime(df_bertopic_kickstarter_technology['project_launched_at']).apply(lambda x: (datetime.timestamp(x))).tolist()

In [48]:
len(df_bertopic_kickstarter_technology.index)

26976

In [49]:
len(timestamps)

26976

In [50]:
len(docs)

26976

In [54]:
docs[1]

"I've begun prototyping a low-cost LED light for the Kindle. I'd like the finished product to be something that I can offer as a DIY kit for end users to purchase with parts + instructions, and I'll post how to do it on your own without the kit on my website too. So far, I've just been toying with what parts to use, how to position for good light distribution, etc. Here's a flickr set of the first very basic progression of prototyping to date:\r\n I've also posted about the process on my blog:\r\n\r\n For the next steps, I want to add a switch, move the battery to the back of the Kindle, and make the building of the LED light less of a bend and tape operation and a more polished LED kit. In order to make this next step I could use some money to buy a bunch of parts and tools for prototyping (I need a new soldering iron, wires, heat shrink tubing, etc.). If and when I get it to the level where I begin offering the LED kit to the public, I'll give all donors of this successfully funded p

In [59]:
topics_over_time = topic_model.topics_over_time(docs, timestamps, nr_bins=20)

3it [02:18, 46.06s/it]


KeyboardInterrupt: 

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, topics=[9, 10, 72, 83, 87, 91])

### Topic document Datafrem

In [ ]:
document_info = topic_model.get_document_info(docs)